In [1]:
import argparse
import glob

import seaborn as sns
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import datasets, plotting
from statsmodels.stats.multitest import multipletests
from src import custom_plotting as cp
from nilearn import surface
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
def load_noise_ceiling(mask_dir, sid, mask):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_set-test_stat-rho_statmap.npy')
    inds = np.where(noise_ceiling <= 0.)[0]
    noise_ceiling[inds] = np.nan
    inds = np.where(mask)[0]
    noise_ceiling = noise_ceiling[inds]
    return noise_ceiling

In [3]:
def load_prediction(stat_dir, file_name, mask, mask_im):
    base = f'{stat_dir}/{file_name}'
    rs = np.load(f'{base}_rs.npy')
    ps = np.load(f'{base}_ps.npy')
    return rs, ps

In [4]:
process = 'PlotVoxelEncodingTest'

mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
annotation_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw/annotations'
stat_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/VoxelPermutationTest'
figure_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures'
figure_dir = f'{figure_dir}/{process}'
Path(figure_dir).mkdir(parents=True, exist_ok=True)

mask_im = nib.load(f'{mask_dir}/sub-all_set-test_stat-rho_statmap.nii.gz')
mask = np.load(f'{mask_dir}/sub-all_set-test_reliability-mask.npy')

fsaverage = datasets.fetch_surf_fsaverage()
cmap = sns.color_palette('magma', as_cmap=True)

roi_parcel = None
vmax = 1.

In [5]:
# for sid in ['01', '02', '03', '04']:
sid = '02'
noise_ceiling = load_noise_ceiling(mask_dir, sid, mask)
for model in ['social', 'visual', 'socialprimitive']:
    file_name = f'sub-{sid}_model-full_predict-{model}_control-conv2'

    rs, ps = load_prediction(stat_dir, file_name, mask, mask_im)
    _, ps, _, _ = multipletests(ps, method='fdr_bh')
    normed_rs = rs / noise_ceiling
    normed_rs[normed_rs > 1] = 1.
    normed_rs[ps > 0.05] = 0.
    # threshold = np.nanmin(normed_rs[normed_rs > 0])

    volume = cp.mkNifti(normed_rs, mask, mask_im)
    texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left'], interpolation='nearest'),
               'right': surface.vol_to_surf(volume, fsaverage['pial_right'], interpolation='nearest')}

    cp.plot_surface_stats(fsaverage, texture,
                          roi=roi_parcel,
                          cmap=cmap,
                          modes=['lateral', 'ventral'],
                          output_file=f'{figure_dir}/{file_name}.png',
                          threshold=0.001,
                          vmax=1.)